In [1]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dropout, ConvLSTM2D
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utilities_v2 import *
from IFSSNet_utilities_v2 import *

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def create_model(input_shape_x, input_shape_y, num_classes, keep_prob_skip=0.2):   
    x_target = tf.keras.Input(shape=input_shape_x, name='x_target')
    y_estimated = tf.keras.Input(shape=input_shape_y, name='y_estimated')
    print(x_target)
    print(y_estimated)

    ################# Layer 1
    Layer_1 = Conv3DBlock(x_target, n_filters=30, kernel_size=(3, 3, 3), stride=(1, 1, 1), activation_fn=None)
    Layer_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same', name='Layer_1p')(Layer_1)
    Layer_1 = AtrousSpatialPyramidPoolingModule_3D(Layer_1, depth=6)
    Layer_1 = Dropout(rate=1 - keep_prob_skip)(Layer_1)
    print(Layer_1)

    ################ Layer 2
    Layer_2 = Conv3DBlock(Layer_1, 30, kernel_size=(3, 3, 3), stride=(1, 1, 1), activation_fn=None)
    Layer_2 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same', name='Layer_2p')(Layer_2)
    Layer_2 = AtrousSpatialPyramidPoolingModule_3D(Layer_2, depth=6)
    Layer_2 = Dropout(rate=1 - keep_prob_skip)(Layer_2)
    print(Layer_2)

    ############### Layer 3
    Layer_3 = Conv3DBlock(Layer_2, 60, kernel_size=(3, 3, 3), stride=(1, 1, 1), activation_fn=None)
    Layer_3 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same', name='Layer_3p')(Layer_3)
    Layer_3 = AtrousSpatialPyramidPoolingModule_3D_rate_11(Layer_3, depth=12)
    Layer_3 = Dropout(rate=1 - keep_prob_skip)(Layer_3)
    print(Layer_3)
    
    ############## Layer 4
    Layer_4 = Conv3DBlock(Layer_3, 120, kernel_size=(3, 3, 3), stride=(1, 1, 1), activation_fn=None)
    Layer_4 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same', name='Layer_4p')(Layer_4)
    Layer_4 = AtrousSpatialPyramidPoolingModule_3D_rate_9(Layer_4, depth=24)
    Layer_4 = Dropout(rate=1 - keep_prob_skip)(Layer_4)
    print(Layer_4)
    
    ############## Layer 5

    Layer_5 = Conv3DBlock(Layer_4, 120, kernel_size=(3, 3, 3), stride=(1, 1, 1), activation_fn=None)
    Layer_5 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same', name='Layer_5p')(Layer_5)
    Layer_5 = AtrousSpatialPyramidPoolingModule_3D_rate_9(Layer_5, depth=24)
    print(Layer_5)                                                      
    
    # # Encoder Stream for Estimated Predictions (reuse features from x)

    ################# Layer 1
    Layer_1y = Conv3DBlock(y_estimated, 30, kernel_size=(3,3,3), stride=(1,1,1), activation_fn=None)
    Layer_1y = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(Layer_1y)
    Layer_1y = AtrousSpatialPyramidPoolingModule_3D(Layer_1y, depth=6)
    Layer_1y = Dropout(rate=1 - keep_prob_skip)(Layer_1y)
    print(Layer_1y)

    ############### Layer 2
    Layer_2y = Conv3DBlock(Layer_1y, 30, kernel_size=(3,3,3), stride=(1,1,1), activation_fn=None)
    Layer_2y = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(Layer_2y)
    Layer_2y = AtrousSpatialPyramidPoolingModule_3D(Layer_2y, depth=6)
    Layer_2y = Dropout(rate=1 - keep_prob_skip)(Layer_2y)
    print(Layer_2y)

    ############## Layer 3
    Layer_3y = Conv3DBlock(Layer_2y, 60, kernel_size=(3,3,3), stride=(1,1,1), activation_fn=None)
    Layer_3y = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(Layer_3y)
    Layer_3y = AtrousSpatialPyramidPoolingModule_3D_rate_11(Layer_3y, depth=12)
    Layer_3y = Dropout(rate=1 - keep_prob_skip)(Layer_3y)
    print(Layer_3y)
    
    ############## Layer 4
    Layer_4y = Conv3DBlock(Layer_3y, 120, kernel_size=(3,3,3), stride=(1,1,1), activation_fn=None)
    Layer_4y = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(Layer_4y)
    Layer_4y = AtrousSpatialPyramidPoolingModule_3D_rate_9(Layer_4y, depth=24)
    Layer_4y = Dropout(rate=1 - keep_prob_skip)(Layer_4y)
    print(Layer_4y)
    
    ############# Layer 5
    Layer_5y = Conv3DBlock(Layer_4y, 120, kernel_size=(3,3,3), stride=(1,1,1), activation_fn=None)
    Layer_5y = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 2, 2), padding='same')(Layer_5y)
    Layer_5y = AtrousSpatialPyramidPoolingModule_3D_rate_9(Layer_5y, depth=24)
    print(Layer_5y)
    
    # # Fusion and SpatioTemporal Correlations

    Fused_x_y = FeatureFusionModule(Layer_5, Layer_5y, 120)
    print(Fused_x_y)

    BiSpatioTemporal_x_y_F = ConvLSTM2D(filters=120, kernel_size=(1, 1), padding='same', return_sequences=True, 
                                        go_backwards=False, kernel_initializer='he_normal',
                                        recurrent_dropout=0.3, activation=tf.nn.tanh)(Fused_x_y)
    print(BiSpatioTemporal_x_y_F)

    Fused_x_y_Reversed = Lambda(lambda x: tf.reverse(x, axis=[1]))(Fused_x_y)
    print(Fused_x_y_Reversed)

    BiSpatioTemporal_x_y_B = ConvLSTM2D(filters=120, kernel_size=(1, 1), padding='same', return_sequences=True, 
                                        go_backwards=False, kernel_initializer='he_normal',
                                        recurrent_dropout=0.3, activation=tf.nn.tanh)(Fused_x_y_Reversed)
    print(BiSpatioTemporal_x_y_B)

    # Combine forward and backward passes
    BiSpatioTemporal_x_y = Add()([BiSpatioTemporal_x_y_F, Lambda(lambda x: tf.reverse(x, axis=[1]))(BiSpatioTemporal_x_y_B)])
    print(BiSpatioTemporal_x_y)

    # # Build the Decoder

    # Building the Decoder Layers

    ########## Layer 6
    Layer_6 = Conv3DBlockTranspose(BiSpatioTemporal_x_y, 120, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_6 = Conv3DBlock(Layer_6, 120, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_6 = FeatureFusionModule(input_1=Layer_6, input_2=BiSpatioTemporal_x_y, n_filters=120)
    Layer_6 = unpool_3D(Layer_6)
    Layer_6 = layers.Dropout(1 - keep_prob_skip)(Layer_6)
    print(Layer_6)
    
    ######### Layer 7
    Layer_7 = Conv3DBlockTranspose(Layer_6, 120, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_7 = Conv3DBlock(Layer_7, 120, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_7 = FeatureFusionModule(input_1=Layer_7, input_2=Layer_4, n_filters=120)
    Layer_7 = unpool_3D(Layer_7)
    Layer_7 = layers.Dropout(1 - keep_prob_skip)(Layer_7)
    print(Layer_7)
    
    ######### Layer 8
    Layer_8 = Conv3DBlockTranspose(Layer_7, 60, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_8 = Conv3DBlock(Layer_8, 60, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_8 = FeatureFusionModule(input_1=Layer_8, input_2=Layer_3, n_filters=60)
    Layer_8 = unpool_3D(Layer_8)
    Layer_8 = layers.Dropout(1 - keep_prob_skip)(Layer_8)
    print(Layer_8)
        
    ######## Layer 9
    Layer_9 = Conv3DBlockTranspose(Layer_8, 60, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_9 = Conv3DBlock(Layer_9, 60, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_9 = FeatureFusionModule(input_1=Layer_9, input_2=Layer_2, n_filters=60)
    Layer_9 = unpool_3D(Layer_9)
    Layer_9 = layers.Dropout(1 - keep_prob_skip)(Layer_9)
    print(Layer_9)
    
    ######## Layer 10
    Layer_10 = Conv3DBlockTranspose(Layer_9, 16, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_10 = Conv3DBlock(Layer_10, 16, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_10 = FeatureFusionModule(input_1=Layer_10, input_2=Layer_1, n_filters=16)
    Layer_10 = unpool_3D(Layer_10)
    Layer_10 = layers.Dropout(1 - keep_prob_skip)(Layer_10)
    print(Layer_10)
    
    ######## Layer 11
    Layer_11 = Conv3DBlockTranspose(Layer_10, 8, kernel_size=[3,3,3], stride=[1,1,1], activation_fn=None)
    Layer_11 = Conv3DBlock(Layer_11, 8, kernel_size=[1,1,1], stride=[1,1,1], activation_fn=None)
    Layer_11 = layers.Dropout(1 - keep_prob_skip)(Layer_11)  
    print(Layer_11)     

    net_output_256_logits = Conv3D(num_classes, kernel_size=(1,1,1), activation=tf.nn.relu)(Layer_11)
    print(net_output_256_logits)

    net_output_256 = pixel_wise_softmax(net_output_256_logits)
    print(net_output_256)

    model = Model(inputs=[x_target, y_estimated], outputs= net_output_256)
    return model

time_step = None # whatever the depth of the volume, we use a sliding window of T
H=512
W=512
C = 1 # number of input channels
num_classes = 2 # related to SOl or GL or GM muscles
n_class = 2

input_shape_x = (time_step, H, W, C)
input_shape_y = (time_step, H, W, num_classes)
model = create_model(input_shape_x, input_shape_y, num_classes)
model.compile(optimizer='adam', loss='tversky', metrics=['accuracy']) # paper loss = Tversky loss
#model.summary()

In [ ]:
mask_directory = 'C:/Users/carla/Documents/Master_Thesis/Example_Images/FALLMUD/NeilCronin/training_fascicle_masks_small/'

image_directory = 'C:/Users/carla/Documents/Master_Thesis/Example_Images/FALLMUD/NeilCronin/training_images_small/'

sub_vol_size = 5
last_ind = sub_vol_size-1

image_dataset = []
for path in glob.glob(image_directory):
    for img_path in glob.glob(os.path.join(path, "*.tif")):
        img = cv2.imread(img_path, 1)
        img = cv2.resize(img, (512,512))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.reshape((img.shape[0], img.shape[1], 1))
        img = np.array(img)
        img = img/255.0
        image_dataset.append(img)  
image_dataset = np.array(image_dataset)

stacked_image_dataset = []
for i in range(len(image_dataset)-last_ind):
    stacked_images = tf.stack([image_dataset[i], image_dataset[i+1], image_dataset[i+2]], axis=0)
    stacked_image_dataset.append(stacked_images)
stacked_image_dataset = np.array(stacked_image_dataset)

mask_dataset = []
for path in glob.glob(mask_directory):
    for mask_path in glob.glob(os.path.join(path, "*.tif")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (512,512))
        mask = np.stack((mask, mask), axis=-1)
        mask = np.array(mask)
        mask = mask/255.0
        mask_dataset.append(mask)        
mask_dataset = np.array(mask_dataset)

stacked_mask_dataset = []
for i in range(len(mask_dataset)-last_ind):
    stacked_mask = tf.stack([mask_dataset[i], mask_dataset[i+1], mask_dataset[i+2]], axis=0)
    stacked_mask_dataset.append(stacked_mask)
stacked_mask_dataset = np.array(stacked_mask_dataset)

x_train = stacked_image_dataset
y_train = stacked_mask_dataset

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

checkpoint = ModelCheckpoint(f'K-IFSS-Net-simple.keras', verbose=1, save_best_only=True, save_weights_only=False)
early_stopping = EarlyStopping(patience=8, verbose=1)

batch_size = 2
epochs = 2

In [ ]:
history = model.fit([x_train, y_train], y_train,
                    validation_data=([x_val, y_val], y_val),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint, early_stopping])